In [1]:
"""
This notebook can be used to perform a bell test and calculate the S value between each pair of output ports
"""
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController 
from src.kinetic_mount_controller import KineticMountControl

# Setup SPCMs

In [2]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

In [3]:
# set alice channels
TTC.set_alice_reflection_channel(3)
TTC.set_alice_transmission_channel(2)

# set bob channels
TTC.set_bob_reflection_channel(4)
TTC.set_bob_transmission_channel(1)

# NOTE Verify that each trace is correctly labelled and then stop and close the graphic again
#TTC.displayCountTraces()

In [4]:
#TTC.displayCoincidenceTraces()

# Setup Kinetic Mounts

In [5]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=3)


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9', 'COM14']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount)
Device succesfully connected (Dual-Position Slider)
Device succesfully connected (Rotation Mount)


In [6]:
# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(6)
KMC.alice.home()
# make sure shutter is open
KMC.shutter.open()

2

In [7]:
TTC.measureS(KMC, CHSH_angles=[0.0, -44.99765161562661, -22.49651806456107, 22.50037066640107])

[1 3 0 0]
[1 1 1 1]
corr[0,0] = 0.2
	N[0]=1
	N[1]=4
	N[2]=0
	N[3]=5
corr[0,1] = -0.75
	N[0]=0
	N[1]=4
	N[2]=3
	N[3]=1
corr[1,0] = -0.2
	N[0]=0
	N[1]=3
	N[2]=0
	N[3]=2
corr[1,1] = 1.0
	N[0]=1
	N[1]=0
	N[2]=0
	N[3]=2
S = 0.6499999999999999
